In [1]:
!git clone https://github.com/markkirby95/LS-Hackathon.git

Cloning into 'LS-Hackathon'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), done.
